****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
sample_json_df = pd.read_json('/Users/tancy/dropbox/iPythonFolder/data_wrangling_json/data/world_bank_projects.json')
sample_json_df.head(3)

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en
2,{'$oid': '52b213b38594d8a2be17c782'},2014,November,2013-11-01T00:00:00Z,MINISTRY OF FINANCE AND ECONOMIC DEVEL,NaN,Tuvalu!$!TV,TV,Tuvalu,Tuvalu,...,TI,IBRD,Active,Y,"{'Name': 'Regional integration', 'Percent': 46}","[{'name': 'Regional integration', 'code': '47'...","52,81,25,47",6060000,6060000,http://www.worldbank.org/projects/P145310?lang=en


In [ ]:
#sample_json_df.columns

In [3]:
# extract necessary columns
sample_countries = sample_json_df[['countrycode','countryname','themecode']]

# check whether blank in themecode
sample_countries.empty

False

In [4]:
# split multiple values of cell in themecode for each country
newthemecode = sample_countries.themecode.str.split(',').apply(pd.Series, 1).stack()
newthemecode.index = newthemecode.index.droplevel(-1)

# merge new sample_countries and newthemecode
countrytheme = sample_countries.join(pd.DataFrame(newthemecode))

# rename the new column
countrytheme = countrytheme.rename(columns={0: 'newthemecode'})

# count the number of themes of countries
countrythemeCount = countrytheme.groupby('countryname')[['newthemecode']].count()

# select most themes
countrythemeCount.sort_values(by ='newthemecode', ascending = False).head(10)


,newthemecode
countryname,
Republic of Indonesia,52
Republic of India,50
Federative Republic of Brazil,41
People's Republic of Bangladesh,40
Africa,38
Socialist Republic of Vietnam,34
People's Republic of China,32
Kingdom of Morocco,30
Republic of Yemen,30


****
Find the top 10 major project themes (using column 'mjtheme_namecode')
In 2. above you will notice that some entries have only the code and the name is missing. 
Create a dataframe with the missing names filled in.

In [5]:
# # get mjtheme_namecode as DataFrame
Stheme = pd.DataFrame(sample_json_df.mjtheme_namecode)

# # split series in mjtheme_namecode
Stheme1 = Stheme['mjtheme_namecode'].apply(pd.Series)

#converting DataFrames from "wide" to "long" format
Stheme_long = pd.melt(Stheme1) 

In [6]:
# check whether NaN in data
Stheme_long.isnull().any()


# remove row contains NAN
Stheme_long = Stheme_long.dropna(subset=['value'])
Stheme_long.head()


,variable,value
0,0,"{'name': 'Human development', 'code': '8'}"
1,0,"{'name': 'Economic management', 'code': '1'}"
2,0,"{'name': 'Trade and integration', 'code': '5'}"
3,0,"{'name': 'Social dev/gender/inclusion', 'code'..."
4,0,"{'name': 'Trade and integration', 'code': '5'}"


In [7]:
# Create dataframe where to store the dictionary values
ThemeDat = pd.DataFrame(columns=['Name','Code'])

# Populate ThemeDat dataframe
for (i,r) in Stheme_long.iterrows():
    e = r['value']
    ThemeDat.loc[i] = [e['name'], e['code']]


In [12]:
# fill empty str as nan
ThemeDat = ThemeDat.apply(lambda x: x.str.strip()).replace('', np.nan)
# sort Code
ThemeDat = ThemeDat.sort_values('Code')
#ThemeDat.head()

In [10]:
# make a column of first values for each group
x = ThemeDat['Code'].map(ThemeDat.groupby('Code')['Name'].first())

# fill only NaN values using new column x
ThemeDat['Name'] = ThemeDat['Name'].fillna(x)

print(ThemeDat.isnull().any())

#ThemeDat.head()


Name    False
Code    False
dtype: bool


In [11]:
ThemeDatSum = ThemeDat.groupby('Name')['Code'].count()

# get most 10 themes
ThemeDatSum.sort_values(ascending=False).head(10)


Name
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: Code, dtype: int64